https://ollama.com/  
https://github.com/ollama/ollama?tab=readme-ov-file  

1 - ollama 安装   
- https://github.com/ollama/ollama?tab=readme-ov-file
- https://github.com/ollama/ollama/blob/main/docs/linux.md  
- https://blog.csdn.net/u010197332/article/details/137604798?spm=1001.2014.3001.5506  

2 - ollama 配置大模型目录（Windows）  
- 打开 Windows 系统设置，来到【关于】，在右边找到【高级系统设置】，然后会出现一个【系统属性】窗口  
- 在上面的标签里选择【高级】，再点击下方的【环境变量】  
- 这时弹出一个【环境变量】窗口，点击【系统变量】下的新建，然后又弹出一个窗口  
- 变量名：OLLAMA_MODELS  
- 变量值：F:\LLM\ollama  

3 - ollama 配置大模型目录（Linux）  
- vim ~/.bashrc  
- export OLLAMA_MODELS=/home/Ollama/ollama_cache  
- cp -r /usr/share/ollama/.ollama/models /home/Ollama/ollama_cache  

4 - ollama 模型微调  
- https://blog.csdn.net/spiderwower/article/details/138755776?ops_request_misc=%257B%2522request%255Fid%2522%253A%25226fa4e7737d516cd7af7ea785e76a75a1%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=6fa4e7737d516cd7af7ea785e76a75a1&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~top_positive~default-1-138755776-null-null.142^v101^pc_search_result_base5&utm_term=ollama%20%E5%BE%AE%E8%B0%83&spm=1018.2226.3001.4187

4 - ollama 常见命令  
- ollama --version   
- ollama serve  
- ollama create /path/to/Modelfile  
- ollama show my_model  
- ollama run my_model  
- ollama list  
- ollama ps  
- ollama cp my_model new_model  
- ollama rm my_model  
- ollama --help  
- ollama run --help  

In [9]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import numpy as np
import pandas as pd
import torch as th
import ollama

from pprint import pp
from IPython.display import (Markdown, display)
from dotenv import load_dotenv
from datasets import (load_dataset, load_from_disk, Dataset)
from transformers import (AutoTokenizer, 
                          BitsAndBytesConfig,
                          AutoModel, 
                          AutoModelForCausalLM, 
                          AutoModelForSequenceClassification,
                          DataCollatorWithPadding, 
                          DataCollatorForLanguageModeling,
                          DataCollatorForSeq2Seq, 
                          DataCollatorForTokenClassification,
                          TrainingArguments, Trainer,
                          pipeline)
from peft import (LoraConfig, get_peft_model, PeftModel, TaskType, get_peft_model_state_dict)
from trl import SFTTrainer
from openai import OpenAI
from ollama import Client

In [10]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [11]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [53]:
load_dotenv(dotenv_path="explore.env")
baidu_key = os.getenv("BAIDU_KEY")

## step-2: 实例化客户端

In [25]:
client = Client(
    host="http://127.0.0.1:11434",
    timeout=10
)

## step-3: 模型推理

In [31]:
checkpoint = "qwen2.5:7b"

In [32]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)

In [33]:
user_prompt = "你好呀~"

In [34]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [44]:
gen_kwargs = {
    "max_new_tokens": 512,
    "do_sample": True,
    "num_beams": 2,
    "temperature": 1.5,
    "top_p": 0.9,
}

response = client.chat(
    model=checkpoint,
    messages=messages,
    options=gen_kwargs,
    stream=False
)

In [45]:
pp(response)

{'model': 'qwen2.5:7b',
 'created_at': '2025-01-29T13:32:25.3973661Z',
 'message': {'role': 'assistant',
             'content': '你好！很高兴为您服务。有什么关于差旅的问题或者需要帮助的吗？无论是酒店预订、行程规划还是其他相关事宜，都欢迎您来询问。'},
 'done_reason': 'stop',
 'done': True,
 'total_duration': 1078046400,
 'load_duration': 21736700,
 'prompt_eval_count': 94,
 'prompt_eval_duration': 3000000,
 'eval_count': 35,
 'eval_duration': 1046000000}


In [46]:
display(Markdown(response["message"]["content"]))

你好！很高兴为您服务。有什么关于差旅的问题或者需要帮助的吗？无论是酒店预订、行程规划还是其他相关事宜，都欢迎您来询问。

In [ ]:
# stream
response = client.chat(
    model=checkpoint,
    messages=messages,
    options=gen_kwargs,
    stream=True
)
for res in response:
    print(res["message"]["content"])

## step-4: 工具调用

In [50]:
cityName2districtId = {
    "南京": "320100",
    "深圳": "440300"
}

In [54]:
def get_weather(cityName):
    districtId = cityName2districtId.get(cityName)
    url = f"https://api.map.baidu.com/weather/v1/?district_id={districtId}&data_type=all&ak={baidu_key}"
    response = requests.get(url)
    data = response.json()
    return json.dumps(data)

In [55]:
# test tool
data = get_weather(cityName="南京")
data

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 4, "feels_like": 2, "rh": 36, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u5357\\u98ce", "uptime": "20250129213000"}, "forecasts": [{"text_day": "\\u6674", "text_night": "\\u6674", "high": 9, "low": 1, "wc_day": "<3\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u4e1c\\u5357\\u98ce", "date": "2025-01-29", "week": "\\u661f\\u671f\\u4e09"}, {"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 2, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u4e1c\\u98ce", "date": "2025-01-30", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u5c0f\\u96e8", "text_night": "\\u4e2d\\u96e8", "high": 9, "low": 5, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~4\\

In [56]:
get_weather_tool = {
    "name": "get_weather",
    "description": "根据输入的城市名称，查询天气",
    "parameters": {
        "type": "object",
        "properties": {
            "cityName": {
                "type": "string",
                "description": "城市名称"
            }
        },
        "required": ["cityName"]
    }
}

In [57]:
tools = [
    {
        "type": "function",
        "function": get_weather_tool
    }
]

In [58]:
tool_dict = {
    "get_weather": get_weather
}

In [59]:
user_prompt = "帮我查下南京的天气"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [60]:
response = client.chat(
    model=checkpoint,
    messages=messages,
    options=gen_kwargs,
    stream=False,
    tools=tools
)

In [64]:
# assistant response
assistant_response = response["message"]
assistant_response

{'role': 'assistant',
 'content': '',
 'tool_calls': [{'function': {'name': 'get_weather',
    'arguments': {'cityName': '南京'}}}]}

In [65]:
messages.append(assistant_response)

In [68]:
# tool response
tool_name = assistant_response["tool_calls"][0]["function"]["name"]
tool_to_call = tool_dict.get(tool_name)
tool_args = assistant_response["tool_calls"][0]["function"]["arguments"]

tool_response = tool_to_call(**tool_args)
tool_response

'{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 4, "feels_like": 2, "rh": 36, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u5357\\u98ce", "uptime": "20250129213500"}, "forecasts": [{"text_day": "\\u6674", "text_night": "\\u6674", "high": 9, "low": 1, "wc_day": "<3\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u4e1c\\u5357\\u98ce", "date": "2025-01-29", "week": "\\u661f\\u671f\\u4e09"}, {"text_day": "\\u591a\\u4e91", "text_night": "\\u9634", "high": 15, "low": 2, "wc_day": "3~4\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u4e1c\\u98ce", "date": "2025-01-30", "week": "\\u661f\\u671f\\u56db"}, {"text_day": "\\u5c0f\\u96e8", "text_night": "\\u4e2d\\u96e8", "high": 9, "low": 5, "wc_day": "3~4\\u7ea7", "wd_day": "\\u5317\\u98ce", "wc_night": "3~4\\

In [69]:
messages.append(
    {
        "role": "tool",
        "content": tool_response
    }
)

In [70]:
messages

[{'role': 'system',
  'content': '你叫小慧助手，是由Lukas开发的差旅智能客服。你的身份是一名差旅秘书，你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。You are a helpful assistant on business travel.'},
 {'role': 'user', 'content': '帮我查下南京的天气'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'function': {'name': 'get_weather',
     'arguments': {'cityName': '南京'}}}]},
 {'role': 'tool',
  'content': '{"status": 0, "result": {"location": {"country": "\\u4e2d\\u56fd", "province": "\\u6c5f\\u82cf\\u7701", "city": "\\u5357\\u4eac\\u5e02", "name": "\\u5357\\u4eac", "id": "320100"}, "now": {"text": "\\u6674", "temp": 4, "feels_like": 2, "rh": 36, "wind_class": "2\\u7ea7", "wind_dir": "\\u4e1c\\u5357\\u98ce", "uptime": "20250129213500"}, "forecasts": [{"text_day": "\\u6674", "text_night": "\\u6674", "high": 9, "low": 1, "wc_day": "<3\\u7ea7", "wd_day": "\\u4e1c\\u5357\\u98ce", "wc_night": "<3\\u7ea7", "wd_night": "\\u4e1c\\u5357\\u98ce", "date": "2025-01-29", "week": "\\u661f\\u671f\\u4e0

In [71]:
second_response = client.chat(
    model=checkpoint,
    messages=messages,
    options=gen_kwargs,
    stream=False
)

In [75]:
display(Markdown(second_response["message"]["content"]))

南京的天气情况如下：

当前温度：4℃，感觉像是稍微有点冷。湿度36%，风速2级，方向为东北偏西。

今天（1月29日）白天和夜晚都是多云的天气，最高温度约为9℃，最低温度在1℃左右；

明天（1月30日）将有小雨出现，夜间可能会有中等强度的降水，最高气温升至15℃，最低气温降至2℃。

请注意随时关注气象信息，适时调整出行和活动安排。如果需要进一步的帮助，请告诉我。